# TP1
## Grupo 15

Carlos Eduardo Da Silva Machado A96936

Gonçalo Manuel Maia de Sousa A97485

## Problema 1

### Descrição do Problema.

Pretende-se construir um horário semanal para o plano de reuniões de projeto de uma “StartUp” de acordo com as seguintes condições:

    1. Cada reunião ocupa uma sala (enumeradas 1...S) durante um “slot”  1..T (hora,dia).  
    
    2.  Cada reunião tem associado um projeto (enumerados 1..P) e um conjunto de participantes. Os diferentes colaboradores são enumerados 1..C.
    
    3. Cada projeto tem associado um conjunto de colaboradores, dos quais um  é o líder. Cada projeto realiza um dado número de reuniões semanais. 
    
    4. O líder do projeto participa em todas as reuniões do seu projeto; os restantes colaboradores podem ou não participar consoante a sua disponibilidade, num mínimo (“quorum”) de  50% do total de colaboradores do projeto. 

### Abordagem do Problema.

Temos como input's do problema as variaveís $P$, $C$, $S$ e $T$, respetivamente, o conjunto de colaboradores, o líder, o número mínimo de reuniões semanais de cada projeto e a disponibilidade de cada participante (representada numa matriz 2x2).

Estamos perante um problema de alocação, e por isso, será necessário criar uma familia $x_{p,c,s,t}$ de variáveis binárias, tal que:

$$x_{p,c,s,t} = 1 \quad \mathbf{sse} \quad \mbox{o colaborador $c$ tem reunião sobre o projeto $p$ na sala $s$ no slot $t$} $$.

As variáveis $P\times C \times S \times T$ são apresentadas na matriz de alocação $X$ com valores instânciados $\{0,1\}^{P\times C \times S \times T}$.

Além disso, temos também outras duas matrizes $2\times2$, a matriz da disponibilidade ($P \times C$) e a matriz dos colaboradores ($C \times T$).

Na matriz da disponibilidade:
$$d_{c,t} = 1 \quad \mathbf{sse} \quad \mbox{o colaborador $c$ tem disponiblidade no slot $t$} $$.

Na matriz dos colaboradores:
$$d_{p,c} = 1 \quad \mathbf{sse} \quad \mbox{o projeto $p$ tem $c$ como colaborador} $$.

Além das matrizes, temos dois dicionários, um com o número mínimo de reuniões semanais ($m$) e outro com os lideres de cada projeto ($l$).

Adicionalmente, establelecemos as seguintes restrições para o $\textit{solver}$:

    - O lider participa em todas as reuniões sobre o seu projeto;$\nl$
    - O projeto tem um mínimo de 50% do total de colaboradores do projeto;
    - Um colaborador apenas pode ter reuniões em slots nos quais está disponíel;
    - Todos os projetos devem ser alocados um número mínimo de reuniões semanais;
    - Um colaboradore não podem participar numa reunião sobre um projeto ao qual não está associado;
    - Um colaborador não pode participar em duas reuniões ao mesmo tempo;
    - Numa sala não pode haver duas reuinões ao mesmo tempo;

Além disso, temos dois critérios de optimização:

   - Maximizar o número de reuniões efetivamente realizadas;
   - Minimizar o número médio de reuniões por participante.

## Código Python

Vamos utilizar a biblioteca de programação linear presente no $\textit{OR-Tools}$, $\textit{pywraplp}$ e criar um $\textit{solver}$ com o nome de $\textbf{horario}$.

In [1]:
from ortools.linear_solver import pywraplp
horario = pywraplp.Solver.CreateSolver('SCIP')

### Inputs do Problema e Exemplos.

Funções auxiliares:

In [2]:
def createMatrix(d,disp,x,y):
    for c in range(1,x+1):
        for t in range(1,y+1):
            if t in disp[c]:
                d[c,t] = 1
            else:
                d[c,t] = 0
                
def printmatrix(x,m,n):
    for i in range(1,m+1):
        for j in range(1,n+1):
            print(x[i,j], end = ' ')  
        print()
        
def printdict(d):
    for k in d:
        print(f'{k}-{d[k]}; ')

Exemplo 1:

In [3]:
P,C,S,T = 6, 6, 12, 6

d = {}
disp = {
    1:[1,4,6],
    2:[1,2,3,4] + [6],
    3:[2,4,5],
    4:[2,4,5,6],
    5:[2,3,4,6],
    6:[4]
}
createMatrix(d,disp,C,T)
print("Matriz de disponibilidade: ")
printmatrix(d,C,T)
col = {}
colab ={
    1:list(range(1,7)),
    2:[2,4,5],
    3:[6],
    4:[1,2,4],
    5:[1,2,3,4,6],
    6:[1,2,3,4,5]
} 
createMatrix(col,colab,P,C)
print("\nMatriz dos colaboradores: ")
printmatrix(col,P,C)
l = {}
print("\nLíderes: ")
l = {1:1,2:2,3:6,4:4,5:3,6:5}
printdict(l)
print("\nMínimo por projeto: ")
m = {1:1,2:1,3:1,4:1,5:1,6:1}
printdict(m)

Matriz de disponibilidade: 
1 0 0 1 0 1 
1 1 1 1 0 1 
0 1 0 1 1 0 
0 1 0 1 1 1 
0 1 1 1 0 1 
0 0 0 1 0 0 

Matriz dos colaboradores: 
1 1 1 1 1 1 
0 1 0 1 1 0 
0 0 0 0 0 1 
1 1 0 1 0 0 
1 1 1 1 0 1 
1 1 1 1 1 0 

Líderes: 
1-1; 
2-2; 
3-6; 
4-4; 
5-3; 
6-5; 

Mínimo por projeto: 
1-1; 
2-1; 
3-1; 
4-1; 
5-1; 
6-1; 


Exemplo 2:

In [3]:
P,C,S,T = 5, 10, 10, 10

d = {}
disp = {
    1:list(range(2,10)),
    2:list(range(1,9)),
    3:list(range(1,6)) + list(range(8,11)),
    4:list(range(3,10)),
    5:list(range(2,7)) + list(range(8,11)),
    6:list(range(1,8)) + [10],
    7:list(range(2,10)),
    8:list(range(1,10)),
    9:list(range(3,10)),
    10:list(range(1,11)),
}
createMatrix(d,disp,C,T)

col = {}
colab = {
    1:[1,2],
    2:[4,5],
    3:[6,7],
    4:[8,9],
    5:[3,10]
}
createMatrix(col,colab,P,C)


l = {1:1,2:4,3:6,4:8,5:3}

m = {1:1,2:1,3:1,4:1,5:1}

print("Disponibilidade: ")
printmatrix(d,C,T)
print("\nColaboradores: ")
printmatrix(col,P,C)
print("\nLíderes: ")
printdict(l)
print("\nMínimo por projeto: ")
printdict(m)

Disponibilidade: 
0 1 1 1 1 1 1 1 1 0 
1 1 1 1 1 1 1 1 0 0 
1 1 1 1 1 0 0 1 1 1 
0 0 1 1 1 1 1 1 1 0 
0 1 1 1 1 1 0 1 1 1 
1 1 1 1 1 1 1 0 0 1 
0 1 1 1 1 1 1 1 1 0 
1 1 1 1 1 1 1 1 1 0 
0 0 1 1 1 1 1 1 1 0 
1 1 1 1 1 1 1 1 1 1 

Colaboradores: 
1 1 0 0 0 0 0 0 0 0 
0 0 0 1 1 0 0 0 0 0 
0 0 0 0 0 1 1 0 0 0 
0 0 0 0 0 0 0 1 1 0 
0 0 1 0 0 0 0 0 0 1 

Líderes: 
1-1; 
2-4; 
3-6; 
4-8; 
5-3; 

Mínimo por projeto: 
1-1; 
2-1; 
3-1; 
4-1; 
5-1; 


Exemplo 3:

In [5]:
P,C,S,T = 10,20,20,20
d = {}
disp = {
    1:list(range(2,20)),
    2:list(range(1,19)),
    3:list(range(1,8)) + list(range(8,19)),
    4:list(range(3,20)),
    5:list(range(2,17)) + list(range(8,21)),
    6:list(range(1,18)) + [20],
    7:list(range(2,20)),
    8:list(range(1,20)),
    9:list(range(3,20)),
    10:list(range(1,21)),
    11:list(range(2,20)),
    12:list(range(1,19)),
    13:list(range(1,8)) + list(range(8,19)),
    14:list(range(3,20)),
    15:list(range(2,17)) + list(range(8,21)),
    16:list(range(1,18)) + [20],
    17:list(range(2,20)),
    18:list(range(1,20)),
    19:list(range(3,20)),
    20:list(range(1,21))
}

createMatrix(d,disp,C,T)

col = {}
colab = {
    1:[1,2,3],
    2:[4,5,6],
    3:[7,8,9],
    4:[10,11,12],
    5:[13,14,15],
    6:[16,17,18],
    7:[19,20,1],
    8:[4,7,10],
    9:[5,8,11],
    10:[6,9,12]
}

createMatrix(col,colab,P,C)

l = {
    1:2,
    2:6,
    3:8,
    4:11,
    5:14,
    6:17,
    7:20,
    8:7,
    9:5,
    10:9
}

m = {
    1:1,
    2:1,
    3:1,
    4:1,
    5:1,
    6:1,
    7:1,
    8:1,
    9:1,
    10:1
}

print("Disponibilidade: ")
printmatrix(d,C,T)
print("\nColaboradores: ")
printmatrix(col,P,C)
print("\nLíderes: ")
printdict(l)
print("\nMínimo por projeto: ")
printdict(m)

Disponibilidade: 
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 
0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 
0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 
0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 
0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

Colaboradores: 
1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 1 1 

Exemplo 4:

In [3]:
P,C,S,T = 2,4,4,4

d = {}
disp = {
    1:[1,1,1,0],
    2:[1,1,0,1],
    3:[1,0,1,1],
    4:[0,1,1,1]
}
createMatrix(d,disp,C,T)

col = {}
colab = {
    1:[1,2],
    2:[3,4]
}
createMatrix(col,colab,P,C)

l = {1:1,2:3}

m = {1:1,2:2}

print("Disponibilidade: ")
printmatrix(d,C,T)
print("\nColaboradores: ")
printmatrix(col,P,C)
print("\nLíderes: ")
printdict(l)
print("\nMínimo por projeto: ")
printdict(m)

Disponibilidade: 
1 0 0 0 
1 0 0 0 
1 0 0 0 
1 0 0 0 

Colaboradores: 
1 1 0 0 
0 0 1 1 

Líderes: 
1-1; 
2-3; 

Mínimo por projeto: 
1-1; 
2-2; 


Criação da matriz de alocação como matriz de variáveis binárias do $\textit{solver}$:

In [4]:
X = {}
for p in range(1,P+1):
    for c in range(1,C+1):
        for s in range(1,S+1):
            for t in range(1,T+1):
                X[p,c,s,t] = horario.BoolVar(f'X[{p}][{c}][{s}][{t}]')

### Apresentação das restrições

O lider participa em todas as reuniões sobre o seu projeto:

$$
\forall_{p< P,c< C,s< S,t<T }  \quad x_{p,l_p,s,t} \ge x_{p,c,s,t}
$$

In [5]:
for p in range(1,P+1):
    for c in range(1,C+1):
        for s in range(1,S+1):
            for t in range(1,T+1):
                horario.Add(X[p,l[p],s,t] >= X[p,c,s,t])


O projeto tem um mínimo ($\textit{quorum}$) de 50% do total de colaboradores do projeto:
$$
\forall_{p< P,s< S,t< T} \quad \sum_{c< C} x_{p,c,s,t} \ge \frac{\sum_{c< C} col_{p,c}}{2}x_{p,l_p,s,t}
$$

In [6]:
for p in range(1,P+1):
    for s in range(1,S+1):
        for t in range(1,T+1):
            horario.Add(sum([ X[p,c,s,t] for c in range(1,C+1)]) >= sum([ col[p,c] for c in range(1,C+1) ])*X[p,l[p],s,t]/2)
                

Um colaborador apenas pode ter reuniões em slots nos quais está disponíel:
$$
\forall_{c< C,t< T} \quad d_{c,t} = 0 \quad \implies \quad \sum_{p<P,s< S} x_{p,c,s,t} = 0
$$

In [7]:
for c in range(1,C+1):
    for t in range(1,T+1):
        if d[c,t] == 0:
            horario.Add(sum([ X[p,c,s,t] for p in range(1,P+1) for s in range(1,S+1)]) == 0)

Todos os projetos devem ser alocados um número mínimo de reuniões semanais:
$$
\forall_{p< P} \quad \sum_{s< S,t< T} x_{p,l_p,s,t} \ge m_p
$$

In [8]:
for p in range(1,P+1):
    horario.Add(sum([ X[p,l[p],s,t] for s in range(1,S+1) for t in range(1,T+1)]) >= m[p])

Um colaboradore não podem participar numa reunião sobre um projeto ao qual não está associado:
$$
\forall_{p< P,c< C,s< S,t< T} \quad x_{p,c,s,t} \le col_{p,c}
$$

In [9]:
for p in range(1,P+1):
    for c in range(1,C+1):
        for s in range(1,S+1):
            for t in range(1,T+1):
                horario.Add(X[p,c,s,t] <= col[p,c])

Um colaborador não pode participar em duas reuniões ao mesmo tempo:
$$
\forall_{c< C, t< T} \quad\sum_{p< P, s< S} x_{p,c,s,t} \le 1
$$

In [10]:
for c in range(1,C+1):
    for t in range(1,T+1):
        horario.Add(sum([ X[p,c,s,t] for p in range(1,P+1) for s in range(1,S+1)]) <= 1)

Numa sala não pode haver duas reuinões ao mesmo tempo:
$$
\forall_{s< S, t< T} \quad \sum_{p< P} x_{p,l_p,s,t} \leq 1
$$

In [11]:
for s in range(1, S+1):
    for t in range(1,t+1):
        horario.Add(sum([ X[p,l[p],s,t] for p in range(1,P+1)]) <= 1)

### Apresentação das Otimizações

Maximizar o número de reunões efetivamente realizadas ($m1$), minimizando o número médio de reuniões por participante ($\frac{m2}{C}$) de acordo com o $\lambda$, isto é, maximizando
$$(1-\lambda)*m1 - (\lambda*m2)$$

onde m1 = $
\quad\sum_{p< P, s< S, t< T} x_{p,l_p,s,t}$
e
m2 = $\quad\sum_{p< P, c<C, s< S, t< T} x_{p,l_p,s,t}$

In [12]:
lamb = 0.5 # Default

m1 =sum([X[p1,l[p1],s1,t1] for p1 in range(1,P+1) for s1 in range(1,S+1) for t1 in range(1,T+1)])

m2 = sum([X[p2,c2,s2,t2] for p2 in range(1,P+1) for c2 in range(1,C+1) for s2 in range(1,S+1) for t2 in range(1, T+1)])

horario.Maximize((1-lamb)*m1 - (lamb * (m2/C)))

### Resolução dos exemplos

Função para resolver cada um dos exemplos:

In [13]:
def resolve():
    status = horario.Solve()
    option = 1
    if status == pywraplp.Solver.OPTIMAL:
        if option == 0:
            for c in range(1,C+1):
                print(f'colaborador {c}:')
                for p in range(1,P+1):
                    for s in range(1,S+1):
                        for t in range(1,T+1):
                            if (int(X[p,c,s,t].solution_value()) == 1):
                                print(f'reunião do projeto {p} na sala {s} no slot {t}')
                print()
                print()
            
        if option == 1:
            for p in range(1,P+1):
                n = 1
                x = sum([ int(X[p,l[p],s,t].solution_value())for s in range(1,S+1) for t in range(1,T+1)])
                print(f'projeto {p}:')
                print(f'número de reuniões: {x}')
                for c in range(1,C+1):
                    for s in range(1,S+1):
                        for t in range(1,T+1):
                            if (int(X[p,c,s,t].solution_value()) == 1):
                                print(f'{n} - o colaborador {c} tem reunião na sala {s} no slot {t}')
                                n += 1
                print()
                print()
    else:
        print("No solution found")

Exemplo 1:

In [14]:
resolve()

projeto 1:
número de reuniões: 1
1 - o colaborador 1 tem reunião na sala 2 no slot 4
2 - o colaborador 3 tem reunião na sala 2 no slot 4
3 - o colaborador 5 tem reunião na sala 2 no slot 4


projeto 2:
número de reuniões: 1
1 - o colaborador 2 tem reunião na sala 2 no slot 3
2 - o colaborador 5 tem reunião na sala 2 no slot 3


projeto 3:
número de reuniões: 1
1 - o colaborador 6 tem reunião na sala 7 no slot 4


projeto 4:
número de reuniões: 1
1 - o colaborador 2 tem reunião na sala 4 no slot 4
2 - o colaborador 4 tem reunião na sala 4 no slot 4


projeto 5:
número de reuniões: 1
1 - o colaborador 2 tem reunião na sala 8 no slot 2
2 - o colaborador 3 tem reunião na sala 8 no slot 2
3 - o colaborador 4 tem reunião na sala 8 no slot 2


projeto 6:
número de reuniões: 1
1 - o colaborador 2 tem reunião na sala 9 no slot 6
2 - o colaborador 4 tem reunião na sala 9 no slot 6
3 - o colaborador 5 tem reunião na sala 9 no slot 6




Exemplo 2:

In [14]:
resolve()

projeto 1:
número de reuniões: 8
1 - o colaborador 1 tem reunião na sala 1 no slot 2
2 - o colaborador 1 tem reunião na sala 1 no slot 6
3 - o colaborador 1 tem reunião na sala 1 no slot 7
4 - o colaborador 1 tem reunião na sala 2 no slot 8
5 - o colaborador 1 tem reunião na sala 4 no slot 3
6 - o colaborador 1 tem reunião na sala 4 no slot 5
7 - o colaborador 1 tem reunião na sala 4 no slot 9
8 - o colaborador 1 tem reunião na sala 5 no slot 4


projeto 2:
número de reuniões: 7
1 - o colaborador 4 tem reunião na sala 1 no slot 9
2 - o colaborador 4 tem reunião na sala 2 no slot 4
3 - o colaborador 4 tem reunião na sala 3 no slot 3
4 - o colaborador 4 tem reunião na sala 3 no slot 6
5 - o colaborador 4 tem reunião na sala 3 no slot 7
6 - o colaborador 4 tem reunião na sala 3 no slot 8
7 - o colaborador 4 tem reunião na sala 5 no slot 5


projeto 3:
número de reuniões: 8
1 - o colaborador 6 tem reunião na sala 1 no slot 10
2 - o colaborador 6 tem reunião na sala 2 no slot 1
3 - o colabo

Exemplo 3:

In [19]:
resolve()

projeto 1:
número de reuniões: 18
1 - o colaborador 1 tem reunião na sala 1 no slot 15
2 - o colaborador 1 tem reunião na sala 5 no slot 7
3 - o colaborador 1 tem reunião na sala 6 no slot 6
4 - o colaborador 1 tem reunião na sala 6 no slot 18
5 - o colaborador 1 tem reunião na sala 19 no slot 5
6 - o colaborador 1 tem reunião na sala 19 no slot 9
7 - o colaborador 1 tem reunião na sala 20 no slot 17
8 - o colaborador 2 tem reunião na sala 1 no slot 15
9 - o colaborador 2 tem reunião na sala 2 no slot 1
10 - o colaborador 2 tem reunião na sala 3 no slot 2
11 - o colaborador 2 tem reunião na sala 4 no slot 13
12 - o colaborador 2 tem reunião na sala 5 no slot 7
13 - o colaborador 2 tem reunião na sala 5 no slot 8
14 - o colaborador 2 tem reunião na sala 6 no slot 6
15 - o colaborador 2 tem reunião na sala 6 no slot 16
16 - o colaborador 2 tem reunião na sala 6 no slot 18
17 - o colaborador 2 tem reunião na sala 7 no slot 11
18 - o colaborador 2 tem reunião na sala 7 no slot 12
19 - o co

Exemplo 4:

In [14]:
resolve()

No solution found
